# Detailed execution time for cadCAD models

*Danilo Lessa Bernardineli*

---

This notebook shows how you can get detailed info about the execution time by using Python decorators on the minimal P&P model. The strategy is to make use of the decorator as defined in the next block, and use in the most convenient way.

There are two options: one is to make use of the decorator on the function definition, like by wrapping the `p_predator_births` and the `p_prey_births` policies. 

The other option is to update the `partial_state_update_blocks` by a list comprehension, like ``partial_state_update_blocks = [print_time(psub) for psub in partial_state_update_blocks]``. This is useful if you want to check everyone at once.

In [11]:
from time import time
import logging
from functools import wraps
logging.basicConfig(level=logging.DEBUG)

def print_time(f):
  """

  """
  @wraps(f)
  def wrapper(*args, **kwargs):
      # Current timestep
      t = len(args[2])
      t1 = time()
      f_out = f(*args, **kwargs)
      t2 = time()
      text = f"{t}|{f.__name__} output (exec time: {t2 - t1:.2f}s): {f_out}"
      logging.debug(text)
      return f_out
  return wrapper

## Dependences

In [12]:
%%capture
!pip install cadcad 

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

## Definitions

### Initial conditions and parameters

In [14]:
initial_conditions = {
    'prey_population': 100,
    'predator_population': 15
    }

params = {
    "prey_birth_rate": [1.0],
    "predator_birth_rate": [0.01],
    "predator_death_const": [1.0],
    "prey_death_const": [0.03],
    "dt": [0.1] # Precision of the simulation. Lower is more accurate / slower
}

simulation_parameters = {
    'N': 1,
    'T': range(30),
    'M': params
}

### Policies

In [15]:
@print_time
def p_predator_births(params, step, sL, s):
  dt = params['dt']
  predator_population = s['predator_population']
  prey_population = s['prey_population']
  birth_fraction = params['predator_birth_rate'] + np.random.random() * 0.0002
  births =  birth_fraction * prey_population * predator_population * dt
  return {'add_to_predator_population': births}

@print_time
def p_prey_births(params, step, sL, s):
  dt = params['dt']
  population = s['prey_population']
  birth_fraction = params['prey_birth_rate'] + np.random.random() * 0.1
  births =  birth_fraction * population * dt
  return {'add_to_prey_population': births}


def p_predator_deaths(params, step, sL, s):
  dt = params['dt']
  population = s['predator_population']
  death_rate = params['predator_death_const'] + np.random.random() * 0.005
  deaths = death_rate * population * dt
  return {'add_to_predator_population': -1.0 * deaths}


def p_prey_deaths(params, step, sL, s):
  dt = params['dt']
  death_rate = params['prey_death_const'] + np.random.random() * 0.1
  prey_population = s['prey_population']
  predator_population = s['predator_population']
  deaths = death_rate * prey_population * predator_population * dt
  return {'add_to_prey_population': -1.0 * deaths}

### State update functions

In [16]:
def s_prey_population(params, step, sL, s, _input):
    y = 'prey_population'
    x = s['prey_population'] + _input['add_to_prey_population']
    return (y, x)


def s_predator_population(params, step, sL, s, _input):
    y = 'predator_population'
    x = s['predator_population'] + _input['add_to_predator_population']
    return (y, x)

### State update blocks

In [17]:


partial_state_update_blocks = [
    { 
        'policies': {
            'predator_births': p_predator_births,
            'prey_births': p_prey_births,
            'predator_deaths': p_predator_deaths,
            'prey_deaths': p_prey_deaths,
        },
        'variables': {
            'predator_population': s_prey_population,
            'prey_population': s_predator_population
        }
    }
]

### Configuration and Execution

In [18]:
sim_config = config_sim(simulation_parameters)

exp = Experiment()
exp.append_configs(sim_configs=sim_config, 
                   initial_state=initial_conditions,
                   partial_state_update_blocks=partial_state_update_blocks)


from cadCAD import configs
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.local_mode)
executor = Executor(exec_context=exec_context, configs=configs) 
(records, tensor_field, _) = executor.execute() 

n': 14.275350764469767}
DEBUG:root:137|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 2.412025199976302}
DEBUG:root:137|p_prey_births output (exec time: 0.00s): {'add_to_prey_population': 12.816841169520988}
DEBUG:root:138|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 2.265621539615294}
DEBUG:root:138|p_prey_births output (exec time: 0.00s): {'add_to_prey_population': 11.915573833394193}
DEBUG:root:139|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 2.232234007045967}
DEBUG:root:139|p_prey_births output (exec time: 0.00s): {'add_to_prey_population': 11.1913281862834}
DEBUG:root:140|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 2.180254954799276}
DEBUG:root:140|p_prey_births output (exec time: 0.00s): {'add_to_prey_population': 11.405578737811215}
DEBUG:root:141|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 2.058686682416584}
DEBUG:root:141|p

### Results

In [19]:
import plotly.express as px

In [20]:
df = pd.DataFrame(records)

fig = px.line(df,
              x=df.prey_population,
              y=df.predator_population,
              color=df.run.astype(str))

fig.show()